In [1]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 9.9 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 KB 3.5 MB/s eta 0:00:00a 0:00:01


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

# Assuming 'df' is your pandas DataFrame containing the time series data

print("Starting ARIMA training...")

df = pd.read_csv('./data/df_data_filter.csv')
df = df.drop(columns=['Unnamed: 0'])

wq = ['Chla', 'SS', 'Tur']
n_splits = 5
seed = 1

ARIMA_results = []

for wq_name in wq:
    print(f"\n{wq_name} Started")
    
    for first in range(4, 13):
        print(f"nvar {first}")
        print(f"Seed {seed}")
        
        # Define the TimeSeriesSplit
        tscv = TimeSeriesSplit(n_splits=n_splits)

        for i, (train_index, test_index) in enumerate(tscv.split(df)):
            df_train, df_test = df.iloc[train_index], df.iloc[test_index]
            Y_train, Y_test = df_train[wq_name], df_test[wq_name]

            # Differencing to make the series stationary if necessary
            adf_result = adfuller(Y_train)
            d = 0 if adf_result[1] < 0.05 else 1

            print("\n--------------------------------------")
            print(f'Training split: {train_index[0]} to {train_index[-1]}, Testing split: {test_index[0]} to {test_index[-1]}')

            # Correlation-based feature selection
            c = df_train.corr().copy()
            c = c[wq_name][22:113]
            c = abs(c).sort_values(ascending=False)[0:first]
            var = c.index.tolist()

            # Training the ARIMA model
            model = ARIMA(Y_train, order=(5, d, 0))  # Order parameters (p, d, q) can be adjusted
            model_fit = model.fit()

            # Predictions on the training set
            Y_train_pred = model_fit.predict(start=0, end=len(Y_train)-1)
            Y_train_pred[Y_train_pred < 0] = 0.0

            corr_model = np.corrcoef(Y_train, Y_train_pred)[0, 1]
            rmse = mean_squared_error(Y_train, Y_train_pred, squared=False)
            mae = mean_absolute_error(Y_train, Y_train_pred)
            smape = np.mean(2 * (np.abs(Y_train_pred - Y_train)) / (np.abs(Y_train) + np.abs(Y_train_pred)))
            r_squared = 1 - (np.sum((Y_train - Y_train_pred) ** 2) / np.sum((Y_train - np.mean(Y_train)) ** 2))
            adjusted_r_squared = 1 - (1 - r_squared) * (len(Y_train) - 1) / (len(Y_train) - 1)

            print(f'{wq_name}: best_score: {r_squared}, RMSE: {rmse}')

            # Test model on testing data
            Y_test_pred = model_fit.predict(start=len(Y_train), end=len(Y_train)+len(Y_test)-1)
            Y_test_pred[Y_test_pred < 0] = 0.0

            r_squared_test = 1 - (np.sum((Y_test - Y_test_pred) ** 2) / np.sum((Y_test - np.mean(Y_test)) ** 2))
            corr_test = np.corrcoef(Y_test, Y_test_pred)[0, 1]
            rmse_test = mean_squared_error(Y_test, Y_test_pred, squared=False)
            mae_test = mean_absolute_error(Y_test, Y_test_pred)
            smape_test = np.mean(2 * (np.abs(Y_test_pred - Y_test)) / (np.abs(Y_test) + np.abs(Y_test_pred)))
            print(f'{wq_name}: r_squared_test: {r_squared_test}, rmse_test: {rmse_test}, smape_test: {smape_test}')

            # Store results in a DataFrame
            ARIMA_df = pd.DataFrame({
                'WQ': [wq_name], 'nvar': [first], 'var': [var], 'random_state': [seed],
                'best_score': [r_squared],
                'r2': [r_squared], 'adjusted_r2': [adjusted_r_squared], 'corr_model': [corr_model],
                'rmse': [rmse], 'mae': [mae], 'smape': [smape],
                'r2_test': [r_squared_test], 'corr_test': [corr_test],
                'rmse_test': [rmse_test], 'mae_test': [mae_test], 'smape_test': [smape_test],
                'train_index': [train_index], 'test_index': [test_index],
                'Y_train': [Y_train], 'Y_train_pred': [Y_train_pred],
                'Y_test': [Y_test], 'Y_test_pred': [Y_test_pred]
            })

            ARIMA_results.append(ARIMA_df)
            print("--------------------------------------\n")

        print(f"{wq_name} Finished")

print("All ARIMA training finished.")

# Combine all results into a single DataFrame
ARIMA_result = pd.concat(ARIMA_results, ignore_index=True)

# Save results to CSV
ARIMA_result.to_csv("data/ARIMA_results.csv", index=False)

print("Results saved to data/ARIMA_results.csv")


Starting ARIMA training...

Chla Started
nvar 4
Seed 1

--------------------------------------
Training split: 0 to 58, Testing split: 59 to 117


/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Chla: best_score: 0.3430095236455747, RMSE: 6.905578804785371
Chla: r_squared_test: -0.019655064337777084, rmse_test: 3.0697651687818426, smape_test: 0.7839286690124041
--------------------------------------


--------------------------------------
Training split: 0 to 117, Testing split: 118 to 176


/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Chla: best_score: 0.30244944271992436, RMSE: 5.3469141875688635
Chla: r_squared_test: -0.0022766183455757982, rmse_test: 5.205794717079001, smape_test: 0.92734621022723
--------------------------------------


--------------------------------------
Training split: 0 to 176, Testing split: 177 to 235


...

Tur: best_score: 0.17974587200531822, RMSE: 2.7328586384412366
Tur: r_squared_test: -0.05888295293376977, rmse_test: 7.839506273357009, smape_test: 0.6944348234959892
--------------------------------------


--------------------------------------
Training split: 0 to 176, Testing split: 177 to 235
Tur: best_score: -0.051315741347703225, RMSE: 5.230846370729602
Tur: r_squared_test: -1.4576272876027412, rmse_test: 6.140980187827736, smape_test: 0.9396284830003725
--------------------------------------


--------------------------------------
Training split: 0 to 235, Testing split: 236 to 294


/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Tur: best_score: 0.0632645658035963, RMSE: 4.681837703591083
Tur: r_squared_test: -0.07107834752185349, rmse_test: 5.472199654168534, smape_test: 0.4991229218917638
--------------------------------------


--------------------------------------
Training split: 0 to 294, Testing split: 295 to 353


/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/rohinsood/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Tur: best_score: 0.09015930430074481, RMSE: 4.733370041602364
Tur: r_squared_test: -0.009209415887569072, rmse_test: 2.0285948530393703, smape_test: 0.3335243240182976
--------------------------------------

Tur Finished
All ARIMA training finished.
Results saved to data/ARIMA_results.csv


In [7]:
df = pd.read_csv("data/ARIMA_results.csv")

# Assuming df is your DataFrame
filtered_df = df[df['WQ'] == 'SS']

lowest_rmse_entry_chla = filtered_df.loc[filtered_df['rmse_test'].idxmin()]

print('Lowest rmse_test value:', lowest_rmse_entry_chla['rmse_test'])
print('Other properties:', lowest_rmse_entry_chla["Y_test_pred"])


Lowest rmse_test value: 2.4284922512670097
Other properties: 59     3.674849
60     3.609208
61     3.576374
62     3.566674
63     3.556493
64     3.548904
65     3.545585
66     3.544001
67     3.542931
68     3.542367
69     3.542144
70     3.542044
71     3.541995
72     3.541980
73     3.541980
74     3.541984
75     3.541987
76     3.541990
77     3.541993
78     3.541994
79     3.541995
80     3.541996
81     3.541996
82     3.541997
83     3.541997
84     3.541997
85     3.541997
86     3.541997
87     3.541997
88     3.541997
89     3.541997
90     3.541997
91     3.541997
92     3.541997
93     3.541997
94     3.541997
95     3.541997
96     3.541997
97     3.541997
98     3.541997
99     3.541997
100    3.541997
101    3.541997
102    3.541997
103    3.541997
104    3.541997
105    3.541997
106    3.541997
107    3.541997
108    3.541997
109    3.541997
110    3.541997
111    3.541997
112    3.541997
113    3.541997
114    3.541997
115    3.541997
116    3.541997
117    3.54